### Run explicit simulation

In [4]:
from Simulator import Simulator
from ForceField.Forcefield import TIP5P_force_field, TIP3P_force_field, GB_Neck2_force_field, Vacuum_force_field, Vacuum_force_field_plus_custom
from openmmtorch import TorchForce
from openmm import LangevinMiddleIntegrator
from openmm.unit import kelvin, picosecond, picoseconds
import numpy as np

### Setup Simulation

COPY all files in topologies_and_starting_coordinates from doi: [10.3929/ethz-b-000599309](https://doi.org/10.3929/ethz-b-000599309) to [Data/data](../Data/data/) <br>

In [2]:
work_dir = "../" # directory of the repository
pdb_id = 'kasae' # Amino acid sequence
n_interval = 1000 # Interval for saving frames in steps
ns = 0.1 # Nanoseconds to run the simulation for

### Run TIP5P Simulation (TIP3P analog)

In [12]:
sim = Simulator(work_dir=work_dir,pdb_id=pdb_id,run_name=pdb_id)
sim._datahandler._ready_for_usage = False # Disable as box will be added
sim.forcefield = TIP5P_force_field()
sim.add_box()
sim._datahandler._ready_for_usage = True # Enable to allow simulation run
sim.integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
sim.platform = "GPU"
n_steps = ns / 0.002 * 1000
sim.run_simulation(n_steps=n_steps,minimize=True,n_interval=1000)

From Amber
built explicit System PME 1 nm
Platform:  GPU  ready


### Run GBNeck2 Simulation

In [25]:
sim = Simulator(work_dir=work_dir,pdb_id=pdb_id,run_name=pdb_id)
sim.forcefield = Vacuum_force_field()

def get_charges(): # Get charges from Vacuum force field
    topology = sim._datahandler.topology
    charges = np.array([sim._system.getForces()[3].getParticleParameters(i)[0]._value for i in range(topology._numAtoms)])
    gbn2_parameters = np.empty((topology.getNumAtoms(),7))
    gbn2_parameters[:,0] = charges # Charges
    return gbn2_parameters

sim.forcefield = GB_Neck2_force_field(get_charges())
sim.integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
sim.platform = "GPU"
n_steps = ns / 0.002 * 1000
sim.run_simulation(n_steps=n_steps,minimize=True,n_interval=1000)

From Amber
Platform:  GPU  ready


### Run GNN Simulation

Run [Train_models.ipynb](../MachineLearning/Train_models.ipynb) Jupyter notebook before to generate model.

In [6]:
run_model_file = '../MachineLearning/trained_models/GNN3_model_run_kasae.pt'
sim = Simulator(work_dir=work_dir,pdb_id=pdb_id,run_name=pdb_id)
torch_force = TorchForce(run_model_file)
sim.forcefield = Vacuum_force_field_plus_custom(torch_force)
sim.integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
sim.platform = "GPU"
n_steps = ns / 0.002 * 1000
sim.run_simulation(n_steps=n_steps,minimize=True,n_interval=1000)

From Amber
Platform:  GPU  ready
